In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import minimize
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils

# Load and preprocess MNIST data (using only the first 1000 samples)
(x_train, y_train), (_, _) = mnist.load_data()
num_samples = 1000  # Number of samples to use
x_train = x_train[:num_samples].reshape(num_samples, -1) / 255.0
y_train = y_train[:num_samples]

# Load the test data from the CSV file
file_path = 'C:/Users/pc/Documents/train.csv'
test_data = pd.read_csv(file_path, delimiter=',', header=0)

# Ensure the test data is preprocessed in the same way as the training data
test_data = test_data.values / 255.0  # Normalize the data if needed

# Define the number of nearest neighbors (k) and a distance threshold (adjust as needed)
k = 5
distance_threshold = 0.2  # Adjust the threshold as needed

# Compute pairwise Euclidean distances
distances = squareform(pdist(x_train, metric='euclidean'))

# Create the k-NN graph based on distance threshold
adjacency = (distances < distance_threshold).astype(int)

# Create a NetworkX graph from the adjacency matrix
G = nx.Graph(adjacency)

# Normalize the graph (Laplacian normalization)
degree_matrix = np.diag(np.sum(adjacency, axis=1))  # Degree matrix
normalized_adjacency = np.linalg.inv(np.sqrt(degree_matrix)) @ adjacency @ np.linalg.inv(np.sqrt(degree_matrix))

# Define optimization hyperparameters
alpha = 0.1
epsilon = 1e-6
max_iterations = 10

# Define your graph signal matrix `S` based on class labels (or other features)
num_features = 28 * 28
S = np.random.randn(num_samples, num_features)

# Define C matrix based on class assignment (assuming you've created C based on class labels)
class_assignment = y_train
C = np.diag(class_assignment)

# Ensure that S has the correct shape (num_samples, num_features)
num_samples, num_features = S.shape

# Define diagonal_matrix_C based on the number of samples
diagonal_matrix_C = np.diag(np.diag(S))


# Precompute matrices M1 and M2 based on the normalized adjacency matrix
M1 = (np.eye(num_samples) - normalized_adjacency).T @ (np.eye(num_samples) - normalized_adjacency)
M2 = diagonal_matrix_C.T @ diagonal_matrix_C

# Define your optimization objective and gradient functions
def cost_function(S, alpha, M1, M2):
    CS_known = diagonal_matrix_C @ S
    CS_algorithm = diagonal_matrix_C @ S
    diff_CS = CS_known - CS_algorithm
    J = 0.5 * (np.linalg.norm((S - (M1 + 2 * alpha * M2) @ S) @ normalized_adjacency)**2 +
               alpha * np.linalg.norm(diff_CS)**2)
    return J

def gradient(S, alpha, M1, M2):
    CS_known = diagonal_matrix_C @ S
    CS_algorithm = diagonal_matrix_C @ S
    gradient_J = (M1 + 2 * alpha * M2) @ S - CS_known + CS_algorithm
    return gradient_J

# Perform optimization with iterative updates
for iteration in range(max_iterations):
    result = minimize(cost_function, S.flatten(), args=(alpha, M1, M2), method='L-BFGS-B', jac=gradient,
                      options={'maxiter': max_iterations, 'gtol': epsilon})

    # Extract the final optimized signal matrix S and reshape it to its original shape
    optimized_S = result.x.reshape(S.shape)

# Neural Network
# Reshape the data and normalize it
X_train = x_train
X_test = test_data

# Convert labels to one-hot encoding
num_classes = 10
Y_train = np_utils.to_categorical(y_train, num_classes)

# Create a Sequential model
model = Sequential()
model.add(Dense(512, input_shape=(num_features,)))  # Input shape matches the number of features
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes))  # Output layer with num_classes neurons
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, Y_train, batch_size=128, epochs=10, verbose=1)

# Evaluate the model on the test data
class_probabilities = model.predict(X_test)

# Determine the predicted classes based on the highest probability
predicted_classes = np.argmax(class_probabilities, axis=1)


# Determine the predicted classes based on the highest probability
predicted_classes = np.argmax(class_probabilities, axis=1)

# Now you can continue with your evaluation and visualization code
correct_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_indices = np.nonzero(predicted_classes != y_test)[0]

# Visualize correct predictions
plt.figure()
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3, 3, i + 1)
    plt.imshow(X_test[correct].reshape(28, 28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], y_test[correct]))

plt.tight_layout()

# Visualize incorrect predictions
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3, 3, i + 1)
    plt.imshow(X_test[incorrect].reshape(28, 28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], y_test[incorrect]))

plt.tight_layout()

# Make predictions on the test data
results = model.predict(test_data)
results = np.argmax(results, axis=1)
results = pd.Series(results, name="Label")
submission = pd.concat([pd.Series(range(1, len(results) + 1), name="ImageId"), results], axis=1)

# Save the submission to a CSV file
submission.to_csv("submission3.csv", index=False)




ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 784000 is different from 784)